In [1]:
import math
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
import mysql.connector
import geopandas as gpd
from shapely.geometry import Point, Polygon

In [2]:
with open('Weather_stations_canada_from_weatherstats_ca.csv') as f:
    print(f)

<_io.TextIOWrapper name='Weather_stations_canada_from_weatherstats_ca.csv' mode='r' encoding='cp1252'>


In [3]:
df = pd.read_csv('Weather_stations_canada_from_weatherstats_ca.csv', encoding = 'latin')

In [4]:
df

,Station Name,Latitude,Longitude,Elevation,IATA,WMO ID,Climate ID,Starting Date (hourly data),Ending Date (hourly data),Starting Date (daily data),Ending Date (daily data),Associated Location
0,(AE) BOW SUMMIT,51.70,-116.47,2080 m,NaN,NaN,3050PPF,NaN,NaN,1998-02-01,2007-11-01,NaN
1,100 MILE HOUSE,51.65,-121.30,1059 m,NaN,NaN,1165791,NaN,NaN,1970-05-01,1999-11-01,100 Mile House
2,100 MILE HOUSE,51.65,-121.27,930 m,NaN,NaN,1095790,NaN,NaN,1957-06-01,1959-08-01,NaN
3,100 MILE HOUSE 6NE,51.68,-121.22,928 m,NaN,NaN,1165793,NaN,NaN,1987-06-01,+,100 Mile House
4,108 MILE HOUSE,51.75,-121.33,957 m,NaN,NaN,1095796,NaN,NaN,1970-05-01,1973-07-01,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
9455,ZELMA,51.80,-105.93,541 m,NaN,NaN,4059220,NaN,NaN,1981-06-01,1989-11-01,NaN
9456,ZHODA,49.25,-96.52,305 m,NaN,NaN,5023370,NaN,NaN,1980-11-01,2000-06-01,NaN
9457,ZURICH,43.40,-81.60,257 m,NaN,NaN,6129850,NaN,NaN,1881-07-01,1898-09-01,NaN
9458,ZWEIBRUCKEN A GERMANY,49.22,-7.42,337 m,NaN,NaN,9032000,NaN,NaN,NaN,NaN,NaN


In [5]:
print(len(df['Climate ID'].unique()))

8807


In [6]:
df = df[~(df['Climate ID'].isna())].copy()

In [7]:
df = df[~(df['Elevation'].isna())].copy()

In [8]:
print(len(df['Climate ID'].unique()))

8665


In [9]:
df.reset_index(inplace = True)
df

,index,Station Name,Latitude,Longitude,Elevation,IATA,WMO ID,Climate ID,Starting Date (hourly data),Ending Date (hourly data),Starting Date (daily data),Ending Date (daily data),Associated Location
0,0,(AE) BOW SUMMIT,51.70,-116.47,2080 m,NaN,NaN,3050PPF,NaN,NaN,1998-02-01,2007-11-01,NaN
1,1,100 MILE HOUSE,51.65,-121.30,1059 m,NaN,NaN,1165791,NaN,NaN,1970-05-01,1999-11-01,100 Mile House
2,2,100 MILE HOUSE,51.65,-121.27,930 m,NaN,NaN,1095790,NaN,NaN,1957-06-01,1959-08-01,NaN
3,3,100 MILE HOUSE 6NE,51.68,-121.22,928 m,NaN,NaN,1165793,NaN,NaN,1987-06-01,+,100 Mile House
4,4,108 MILE HOUSE,51.75,-121.33,957 m,NaN,NaN,1095796,NaN,NaN,1970-05-01,1973-07-01,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8660,9454,ZEHNER,50.63,-104.40,683 m,NaN,NaN,4019200,NaN,NaN,1961-08-01,1999-12-01,NaN
8661,9455,ZELMA,51.80,-105.93,541 m,NaN,NaN,4059220,NaN,NaN,1981-06-01,1989-11-01,NaN
8662,9456,ZHODA,49.25,-96.52,305 m,NaN,NaN,5023370,NaN,NaN,1980-11-01,2000-06-01,NaN
8663,9457,ZURICH,43.40,-81.60,257 m,NaN,NaN,6129850,NaN,NaN,1881-07-01,1898-09-01,NaN


In [10]:
df.rename(columns = { 'Station Name' : 'station_name'}, inplace = True)
df.rename(columns = { 'Climate ID' : 'climate_ID'}, inplace = True)
df.columns

Index(['index', 'station_name', 'Latitude', 'Longitude', 'Elevation', 'IATA',
       'WMO ID', 'climate_ID', 'Starting Date (hourly data)',
       'Ending Date (hourly data)', 'Starting Date (daily data)',
       'Ending Date (daily data)', 'Associated Location'],
      dtype='object')

In [11]:
car = gpd.read_file("lcar000a16a_e.shp")
car

,CARUID,CARENAME,CARFNAME,PRUID,PRNAME,geometry
0,1205,Agricultural Region 5,Région agricole 5,12,Nova Scotia / Nouvelle-Écosse,"POLYGON ((8525489.391 1790548.500, 8525639.114..."
1,1301,Agricultural Region 1,Région agricole 1,13,New Brunswick / Nouveau-Brunswick,"POLYGON ((7998578.391 1612051.631, 7998668.814..."
2,1302,Agricultural Region 2,Région agricole 2,13,New Brunswick / Nouveau-Brunswick,"POLYGON ((8204574.134 1544487.454, 8208027.917..."
3,1303,Agricultural Region 3,Région agricole 3,13,New Brunswick / Nouveau-Brunswick,"POLYGON ((8217369.071 1620627.260, 8217713.531..."
4,1304,Agricultural Region 4,Région agricole 4,13,New Brunswick / Nouveau-Brunswick,"POLYGON ((8188457.820 1707919.583, 8188444.194..."
5,2401,Bas-Saint-Laurent,Bas-Saint-Laurent,24,Quebec / Québec,"POLYGON ((7983960.509 1783925.871, 7985523.191..."
6,2402,Saguenay--Lac-Saint-Jean--Côte-Nord,Saguenay--Lac-Saint-Jean--Côte-Nord,24,Quebec / Québec,"MULTIPOLYGON (((8449366.226 2385149.363, 84505..."
7,2403,Québec,Québec,24,Quebec / Québec,"POLYGON ((7811516.777 1616612.783, 7813390.280..."
8,2404,Mauricie,Mauricie,24,Quebec / Québec,"POLYGON ((7459301.751 1597664.723, 7459403.646..."
9,2405,Estrie,Estrie,24,Quebec / Québec,"POLYGON ((7854287.706 1351074.386, 7854803.251..."


In [12]:
car = car.explode()
car.reset_index(inplace = True)
car.drop(columns = ['level_0', 'level_1'], inplace = True)
car

,CARUID,CARENAME,CARFNAME,PRUID,PRNAME,geometry
0,1205,Agricultural Region 5,Région agricole 5,12,Nova Scotia / Nouvelle-Écosse,"POLYGON ((8525489.391 1790548.500, 8525639.114..."
1,1301,Agricultural Region 1,Région agricole 1,13,New Brunswick / Nouveau-Brunswick,"POLYGON ((7998578.391 1612051.631, 7998668.814..."
2,1302,Agricultural Region 2,Région agricole 2,13,New Brunswick / Nouveau-Brunswick,"POLYGON ((8204574.134 1544487.454, 8208027.917..."
3,1303,Agricultural Region 3,Région agricole 3,13,New Brunswick / Nouveau-Brunswick,"POLYGON ((8217369.071 1620627.260, 8217713.531..."
4,1304,Agricultural Region 4,Région agricole 4,13,New Brunswick / Nouveau-Brunswick,"POLYGON ((8188457.820 1707919.583, 8188444.194..."
5,2401,Bas-Saint-Laurent,Bas-Saint-Laurent,24,Quebec / Québec,"POLYGON ((7983960.509 1783925.871, 7985523.191..."
6,2402,Saguenay--Lac-Saint-Jean--Côte-Nord,Saguenay--Lac-Saint-Jean--Côte-Nord,24,Quebec / Québec,"POLYGON ((8449366.226 2385149.363, 8450574.374..."
7,2402,Saguenay--Lac-Saint-Jean--Côte-Nord,Saguenay--Lac-Saint-Jean--Côte-Nord,24,Quebec / Québec,"POLYGON ((7919158.231 2472904.131, 7918578.826..."
8,2402,Saguenay--Lac-Saint-Jean--Côte-Nord,Saguenay--Lac-Saint-Jean--Côte-Nord,24,Quebec / Québec,"POLYGON ((7747962.957 2385157.983, 7748087.914..."
9,2403,Québec,Québec,24,Quebec / Québec,"POLYGON ((7811516.777 1616612.783, 7813390.280..."


In [13]:
car = car.to_crs(epsg=4326)
car

,CARUID,CARENAME,CARFNAME,PRUID,PRNAME,geometry
0,1205,Agricultural Region 5,Région agricole 5,12,Nova Scotia / Nouvelle-Écosse,"POLYGON ((-60.28743 46.95897, -60.28755 46.956..."
1,1301,Agricultural Region 1,Région agricole 1,13,New Brunswick / Nouveau-Brunswick,"POLYGON ((-67.53994 47.51931, -67.53887 47.518..."
2,1302,Agricultural Region 2,Région agricole 2,13,New Brunswick / Nouveau-Brunswick,"POLYGON ((-65.41281 46.25007, -65.39250 46.205..."
3,1303,Agricultural Region 3,Région agricole 3,13,New Brunswick / Nouveau-Brunswick,"POLYGON ((-64.86127 46.82258, -64.86354 46.811..."
4,1304,Agricultural Region 4,Région agricole 4,13,New Brunswick / Nouveau-Brunswick,"POLYGON ((-64.73402 47.63917, -64.74087 47.629..."
5,2401,Bas-Saint-Laurent,Bas-Saint-Laurent,24,Quebec / Québec,"POLYGON ((-66.84872 48.99555, -66.83220 48.985..."
6,2402,Saguenay--Lac-Saint-Jean--Côte-Nord,Saguenay--Lac-Saint-Jean--Côte-Nord,24,Quebec / Québec,"POLYGON ((-57.10761 51.94450, -57.10755 51.923..."
7,2402,Saguenay--Lac-Saint-Jean--Côte-Nord,Saguenay--Lac-Saint-Jean--Côte-Nord,24,Quebec / Québec,"POLYGON ((-63.59906 54.95137, -63.61134 54.948..."
8,2402,Saguenay--Lac-Saint-Jean--Côte-Nord,Saguenay--Lac-Saint-Jean--Côte-Nord,24,Quebec / Québec,"POLYGON ((-66.65114 54.85768, -66.65314 54.851..."
9,2403,Québec,Québec,24,Quebec / Québec,"POLYGON ((-69.84236 48.15431, -69.82015 48.146..."


In [14]:
df['CARUID'] = np.nan

In [15]:
for i in range(0, len(df)):
    station = Point(df.iloc[i].Longitude, df.iloc[i].Latitude)
    for j in range(0, len(car)):
        polygon = car.iloc[j].geometry
        if(station.within(polygon)):
            df.loc[i, 'CARUID'] = car.iloc[j].CARUID
            break



In [16]:
df

,index,station_name,Latitude,Longitude,Elevation,IATA,WMO ID,climate_ID,Starting Date (hourly data),Ending Date (hourly data),Starting Date (daily data),Ending Date (daily data),Associated Location,CARUID
0,0,(AE) BOW SUMMIT,51.70,-116.47,2080 m,NaN,NaN,3050PPF,NaN,NaN,1998-02-01,2007-11-01,NaN,4830
1,1,100 MILE HOUSE,51.65,-121.30,1059 m,NaN,NaN,1165791,NaN,NaN,1970-05-01,1999-11-01,100 Mile House,5905
2,2,100 MILE HOUSE,51.65,-121.27,930 m,NaN,NaN,1095790,NaN,NaN,1957-06-01,1959-08-01,NaN,5905
3,3,100 MILE HOUSE 6NE,51.68,-121.22,928 m,NaN,NaN,1165793,NaN,NaN,1987-06-01,+,100 Mile House,5905
4,4,108 MILE HOUSE,51.75,-121.33,957 m,NaN,NaN,1095796,NaN,NaN,1970-05-01,1973-07-01,NaN,5905
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8660,9454,ZEHNER,50.63,-104.40,683 m,NaN,NaN,4019200,NaN,NaN,1961-08-01,1999-12-01,NaN,4702
8661,9455,ZELMA,51.80,-105.93,541 m,NaN,NaN,4059220,NaN,NaN,1981-06-01,1989-11-01,NaN,4704
8662,9456,ZHODA,49.25,-96.52,305 m,NaN,NaN,5023370,NaN,NaN,1980-11-01,2000-06-01,NaN,4610
8663,9457,ZURICH,43.40,-81.60,257 m,NaN,NaN,6129850,NaN,NaN,1881-07-01,1898-09-01,NaN,3502


In [17]:
connection = mysql.connector.connect(user='root', password='your_password', host='127.0.0.1', database='field_crops')
cursor = connection.cursor()

add_station = ("INSERT INTO weather_station "
               "(climate_ID, CARUID, name, latitude, longitude, elevation) "
               "VALUES (%s, %s, %s, %s, %s, %s)")

#a = 1

for i in range(0, len(df)):

    record = df.iloc[i]
    
    climate_ID = record.climate_ID
    CARUID = record.CARUID
    if ((len(climate_ID) != 7) | (math.isnan(float(CARUID)))):
        continue
        #print(climate_ID)
        #climate_ID = 'AAAAAA' + str(a)
        #a += 1
    name = record.station_name
    latitude = float(record.Latitude)
    longitude = float(record.Longitude)
    elevation = float((record.Elevation.split())[0])
   
    
    data_station = (climate_ID, CARUID, name, latitude, longitude, elevation)
    
    cursor.execute(add_station, data_station)
    connection.commit()


cursor.close()
connection.close()